In [ ]:
import numpy as np

# Задаём матрицу коэффициентов A и вектор правых частей b для системы
# Система уравнений:
#   3x +  2y -   z =  1
#   2x -  2y +  4z = -2
#  -x + 0.5y -   z =  0
A = np.array([
    [3, 2, -1],
    [2, -2, 4],
    [-1, 0.5, -1]
])
b = np.array([1, -2, 0])

# Предполагаемый единственный ответ: [1, -2, -2]

# --- Функция: метод Гаусса (прямой и обратный ход) ---
def gauss_solve(matrix, vector):
    """
    Решает систему линейных уравнений методом Гаусса.
    
    Аргументы:
      matrix (numpy.ndarray): Матрица коэффициентов системы.
      vector (numpy.ndarray): Вектор правых частей.
      
    Возвращает:
      numpy.ndarray: Вектор решений системы.
    
    Вызывает:
      ValueError: Если опорный элемент на диагонали равен нулю или слишком мал.
    """
    A_local = matrix.astype(float)
    b_local = vector.astype(float)
    n = len(b_local)
    
    # Прямой ход: преобразуем матрицу в верхнетреугольный вид
    for pivot in range(n):
        if abs(A_local[pivot, pivot]) < 1e-12:
            raise ValueError(f"Опорный элемент в позиции ({pivot}, {pivot}) равен нулю или слишком мал.")
        for row in range(pivot + 1, n):
            multiplier = A_local[row, pivot] / A_local[pivot, pivot]
            A_local[row, pivot:] -= multiplier * A_local[pivot, pivot:]
            b_local[row] -= multiplier * b_local[pivot]
    
    # Обратный ход: обратная подстановка
    solution = np.zeros(n)
    for i in range(n - 1, -1, -1):
        sum_known = np.dot(A_local[i, i+1:], solution[i+1:])
        solution[i] = (b_local[i] - sum_known) / A_local[i, i]
    
    return solution

# --- Функция: метод Жордана-Гаусса ---
def gauss_jordan_solve(matrix, vector):
    """
    Решает систему линейных уравнений методом Жордана-Гаусса.

    Аргументы:
      matrix (numpy.ndarray): Матрица коэффициентов системы.
      vector (numpy.ndarray): Вектор правых частей.

    Возвращает:
      numpy.ndarray: Вектор решений системы.

    Вызывает:
      ValueError: Если матрица вырождена.
    """
    A_local = matrix.astype(float)
    b_local = vector.astype(float)
    n = len(b_local)
    
    # Формируем расширенную матрицу
    augmented = np.hstack((A_local, b_local.reshape(-1, 1)))
    
    # Прямой и обратный ход: приводим матрицу к редуцированной ступенчатой форме
    for pivot in range(n):
        max_index = pivot + np.argmax(np.abs(augmented[pivot:, pivot]))
        augmented[[pivot, max_index]] = augmented[[max_index, pivot]]
        
        if abs(augmented[pivot, pivot]) < 1e-12:
            raise ValueError("Матрица вырождена: система имеет бесконечное множество решений или не имеет решения.")
        
        augmented[pivot] = augmented[pivot] / augmented[pivot, pivot]
        
        for row in range(n):
            if row != pivot:
                coeff = augmented[row, pivot]
                augmented[row] -= coeff * augmented[pivot]
    
    return augmented[:, -1]

# --- Функция: метод Гаусса с выбором опорного элемента ---
def gauss_elim_with_pivoting(matrix, vector):
    """
    Решает систему линейных уравнений Ax = b методом Гаусса с выбором опорного элемента.
    
    Аргументы:
        matrix (numpy.ndarray): Квадратная матрица коэффициентов.
        vector (numpy.ndarray): Вектор правых частей.
        
    Возвращает:
        numpy.ndarray: Вектор решений системы.
        
    Вызывает:
        ValueError: Если опорный элемент равен нулю или слишком мал.
    """
    A_local = matrix.astype(float)
    b_local = vector.astype(float)
    n = len(b_local)
    
    # Прямой ход с выбором максимального опорного элемента
    for pivot in range(n):
        max_row_index = pivot + np.argmax(np.abs(A_local[pivot:, pivot]))
        A_local[[pivot, max_row_index]] = A_local[[max_row_index, pivot]]
        b_local[pivot], b_local[max_row_index] = b_local[max_row_index], b_local[pivot]
        
        if abs(A_local[pivot, pivot]) < 1e-12:
            raise ValueError("Матрица вырождена или имеет очень малый определитель.")
        
        for row in range(pivot + 1, n):
            scaling_factor = A_local[row, pivot] / A_local[pivot, pivot]
            A_local[row, pivot:] -= scaling_factor * A_local[pivot, pivot:]
            b_local[row] -= scaling_factor * b_local[pivot]
    
    # Обратная подстановка
    solution = np.zeros(n)
    for i in range(n - 1, -1, -1):
        sum_known = np.dot(A_local[i, i+1:], solution[i+1:])
        solution[i] = (b_local[i] - sum_known) / A_local[i, i]
    
    return solution

# --- Функции для LU-разложения и решения системы ---
def decompose_lu(matrix):
    """
    Выполняет LU-разложение матрицы методом Дулитла.
    
    Аргументы:
      matrix (numpy.ndarray): Квадратная матрица для LU-разложения.
    
    Возвращает:
      L (numpy.ndarray): Нижнетреугольная матрица с единицами на диагонали.
      U (numpy.ndarray): Верхнетреугольная матрица.
      
    Вызывает:
      ValueError: Если обнаружен опорный элемент, близкий к нулю.
    """
    A_local = matrix.astype(float)
    n = A_local.shape[0]
    L = np.eye(n, dtype=float)
    U = np.zeros((n, n), dtype=float)
    
    for i in range(n):
        for j in range(i, n):
            total = sum(L[i, k] * U[k, j] for k in range(i))
            U[i, j] = A_local[i, j] - total
        
        if abs(U[i, i]) < 1e-12:
            raise ValueError("Обнаружен нулевой или слишком малый опорный элемент, LU-разложение невозможно.")
        
        for j in range(i + 1, n):
            total = sum(L[j, k] * U[k, i] for k in range(i))
            L[j, i] = (A_local[j, i] - total) / U[i, i]
    
    return L, U

def solve_forward(L, d):
    """
    Решает систему L*y = d методом прямой подстановки.
    
    Аргументы:
      L (numpy.ndarray): Нижнетреугольная матрица с единичной диагональю.
      d (numpy.ndarray): Вектор правых частей.
    
    Возвращает:
      numpy.ndarray: Вектор решения.
    """
    n = L.shape[0]
    y = np.zeros(n, dtype=float)
    
    for i in range(n):
        y[i] = d[i] - sum(L[i, j] * y[j] for j in range(i))
    
    return y

def solve_backward(U, y):
    """
    Решает систему U*x = y методом обратной подстановки.
    
    Аргументы:
      U (numpy.ndarray): Верхнетреугольная матрица.
      y (numpy.ndarray): Вектор, полученный после прямой подстановки.
    
    Возвращает:
      numpy.ndarray: Вектор решения системы.
    """
    n = U.shape[0]
    x = np.zeros(n, dtype=float)
    
    for i in range(n - 1, -1, -1):
        total = sum(U[i, j] * x[j] for j in range(i + 1, n))
        x[i] = (y[i] - total) / U[i, i]
    
    return x

def lu_solve(matrix, vector):
    """
    Решает систему Ax = b с помощью LU-разложения.
    
    Аргументы:
      matrix (numpy.ndarray): Квадратная матрица коэффициентов.
      vector (numpy.ndarray): Вектор правых частей.
    
    Возвращает:
      numpy.ndarray: Вектор решения.
    """
    L, U = decompose_lu(matrix)
    y = solve_forward(L, vector)
    x = solve_backward(U, y)
    return x

# --- Применение всех методов к одной и той же системе ---
print("Используем систему:")
print("A =\n", A)
print("b =", b)
print()

# Метод Гаусса
sol_gauss = gauss_solve(A.copy(), b.copy())
print("Решение (метод Гаусса):", sol_gauss)

# Метод Жордана-Гаусса
sol_gauss_jordan = gauss_jordan_solve(A.copy(), b.copy())
print("Решение (метод Жордана-Гаусса):", sol_gauss_jordan)

# Метод Гаусса с выбором опорного элемента
sol_gauss_pivot = gauss_elim_with_pivoting(A.copy(), b.copy())
print("Решение (Гаусс с выбором опорного элемента):", sol_gauss_pivot)

# LU-разложение
sol_lu = lu_solve(A.copy(), b.copy())
print("Решение (LU-разложение):", sol_lu)
